# Model Training Notebook

This notebooks goes through the process of model training and performance evaluation.

## Used Framework and Modeling Choices

Despite the given low amount of data I have opted for a deep learning approach using [Dart](https://unit8co.github.io/darts/index.html).
Darts offers a number of pre-build deep learning approaches for time series data.
However, in order to meet my project goals I have formed a number of minimun requirements. These include:
- the model supports probabilistic outputs for a probablistic estimation if a given individual might change her behavior
- multivariate input in order to make use of correlations within the data within and across individuals
- support for covariates as behavior is often affected by external factors such as weekends, temperature and others

Based on these requirements I used an implementation of a [Temporal Convolutional Network (TCN)](https://arxiv.org/abs/1906.04397).
This model was preferred over the more classical RNN since it has shown better performance in past competitions. 
Further, it allowed me to try out this new model and to get some experience with its architecture and performance.

In [175]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from typing import List
from functools import reduce
import os
from pathlib import Path

from darts import TimeSeries
from darts.metrics import mape, smape
from darts.models import ExponentialSmoothing, NaiveSeasonal, Prophet, TCNModel, NBEATSModel
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.missing_values import fill_missing_values

from aws_lambda.chalicelib.data_download import S3, Settings

import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data Access and preprocessing

The original data was stored on an AWS S3 bucket to allow access through microservices and continious learning.
Future version should consider a more stable data location, such a dedicated database.

In [176]:
# Download data from S3
TMP_DIR = "/tmp/training_data"
if not os.path.isdir(TMP_DIR):
    os.mkdir(TMP_DIR)

s3 = S3(Settings())
s3.download(TMP_DIR)

In [177]:
files = os.listdir(TMP_DIR)
# this file is ignored since it is not used in the project
csv_files = {k.replace("_summary.csv", ""): pd.read_csv(os.path.join(TMP_DIR, k)) for k in files if k != "weightLog-report.csv"}  
# standardization of data
for key, df in csv_files.items():
    columns = df.columns
    time_column = next(k for k in columns if k.endswith("Hour") or k.endswith("Date") or k == "datetime")
    df.rename(columns={time_column: "datetime"}, inplace=True)
    time_column = "datetime"
    df[time_column] = pd.to_datetime(df[time_column], infer_datetime_format=True)
    df[time_column] = df[time_column].values.astype(np.datetime64)

## Preprocessing

The dataset has two distinct types of data: Daily and hourly records.
In this project I will treat them as two distinct dataset and will not attempt to model the whole dataset together.
However, with some more time a more holistic modeling approach might result in better results.

### Hourly Records

Daily records were modeling together across all subjects.
This was done since daily records were available for similar variables, that is all variables related to exercises.
While there might be no direct correlation between different subjects beyond the influence of common covariates such as time-of-the-day and day-of-the-week, the model might be able to make use of common patters across subjects.



In [178]:
def preprocessing_hourly(dt: pd.DataFrame, scaler: Scaler, variables: List[str], min_req_length: int = 7*24, fill_missing: bool = False):
    idds_timeseries = []
    used_idds = []
    for idd, dd in dt.groupby("Id"):
        n, _ = dd.shape
        if n < min_req_length:
            continue
        if not dd['datetime'].is_unique:
            continue
        if fill_missing:
            _tts = TimeSeries.from_dataframe(dd,"datetime", variables, fill_missing_dates=True, freq="D")
        else:
            _tts = TimeSeries.from_dataframe(dd,"datetime", variables)
        used_idds.append(idd)
        # convert to 32 for faster training
        xdata = _tts.data_array().astype(np.float32)
        _tts = TimeSeries.from_xarray(xdata)
        idds_timeseries.append(_tts)
    print(f"Using {len(idds_timeseries)=}")
    scaled_idds_timeseries = [scaler.fit_transform(k) for k in idds_timeseries]
    return scaled_idds_timeseries, used_idds

def generate_covariates(time_series: List[TimeSeries], attributes: List[str], scalar: Scaler):
    scaler_covariates = scalar
    st_covariates = []
    for ts in time_series:
        _cov = []
        for att in attributes:
            _cov.append(datetime_attribute_timeseries(ts, attribute=att, one_hot=True))
        # stacking
        covariates = _cov.pop(0)
        for c in _cov:
            covariates = covariates.stack(c)
        _scaled = scaler_covariates.fit_transform(covariates)
        # convert to 32 for faster training
        xdata = _scaled.data_array().astype(np.float32)
        _scaled = TimeSeries.from_xarray(xdata)
        st_covariates.append(_scaled)
    return st_covariates

In [179]:
value_scaler = Scaler()
variables = ["TotalIntensity", "TotalSteps", "Calories"]
scaled_idds, used_idds = preprocessing_hourly(csv_files["hourlyActivity"], value_scaler, variables, 7*24)
covar_scaler = Scaler()
st_covariates = generate_covariates(scaled_idds, ['hour', 'weekday'], covar_scaler)

Using len(idds_timeseries)=32


In [180]:
# save the last 72h of each series as test data
train_val = [(k[:-72], k[-72:]) for k in scaled_idds]
cov_train_val = [(k[:-72], k[-72:]) for k in st_covariates]

In [182]:
from darts.utils.likelihood_models import GaussianLikelihood 
model = TCNModel(
    input_chunk_length=72,
    output_chunk_length=6,
    n_epochs=20,
    random_state=0,
    torch_device_str="cuda:0",
    log_tensorboard=True,
    likelihood=GaussianLikelihood(),
    model_name="daily_model", 
    force_reset=True
)

In [183]:
trains, vals = zip(*train_val)
cov_trains, cov_vals = zip(*cov_train_val)

In [184]:
model.fit(trains, past_covariates=cov_trains, verbose=True)

  0%|          | 0/20 [00:00<?, ?it/s]

### Predict ahead

Next I will predict future values and store the relative output back into the S3 bucket.

In [185]:
def generate_future_covarites(dt: pd.DataFrame, freq: str, num: int, attributes: List[str], scaler: Scaler):
    """Generating covariates for future predictions"""
    idx = pd.date_range(dt['datetime'].min(), dt['datetime'].max() + pd.Timedelta(days=20), freq=freq)
    ts = pd.Series(range(len(idx)), index=idx)
    ts = TimeSeries.from_series(ts)
    tts = ts.add_datetime_attribute("weekday")
    tts = tts.add_datetime_attribute("month")
    
    covariates_all = generate_covariates([tts], attributes, scaler)
    _scaled = scaler.fit_transform(covariates_all)
    xdata = _scaled[0].data_array().astype(np.float32)
    _scaled = TimeSeries.from_xarray(xdata)
    covariates_all = [covariates_all[0] for _ in range(num)]
    return covariates_all

In [190]:
all_covariates =  generate_future_covarites(csv_files["hourlyActivity"],"H", len(trains), ['hour', 'weekday'], covar_scaler)
prediction = model.predict(12, vals, past_covariates=all_covariates, num_samples=100)

In [191]:
def save_prediction(prediction, output_name, used_idds, scaler, input_dt):
    # get prediction quantiles
    quantiles = [0.1, 0.5, 0.90]
    new_q_names = ["_lower", "_mean", "_upper"]
    qq_names = {v: k for k, v in zip(new_q_names, quantiles)}

    pred_q = {}
    for q in quantiles:
        pred_q[q] = [scaler.inverse_transform(k.quantile_timeseries(q)) for k in prediction]
        
    # convert to df
    predicted_data = {}
    for idd in range(len(used_idds)):
        pred_qq_idd = []
        for qq, data in pred_q.items():
            _d = data[idd].pd_dataframe().applymap(lambda x: float(x))
            cc = _d.columns
            new_names = [u.replace(f"_{qq}", qq_names[qq]) for u in cc]
            _d.rename(columns={k: v for k, v in zip(cc, new_names)}, inplace=True)
            pred_qq_idd.append(_d)
        predicted_data[used_idds[idd]] = pred_qq_idd
        
    # converting the data to the required fromat for the microservice
    idds_predicted_data = []
    for idd, data_list in predicted_data.items():
        _new_data = pd.concat(data_list, axis=1)
        _new_data['Id'] = str(idd)
        _new_data['datetime'] = _new_data.index
        _new_data['has_prediction'] = 1
        _new_data['has_values'] = 0
        idds_predicted_data.append(_new_data)

    combined_prediction = pd.concat(idds_predicted_data, axis=0, ignore_index=True)
    input_dt['has_prediction'] = 0
    input_dt['has_values'] = 1

    combined_dt = pd.concat([input_dt, combined_prediction], axis=0, ignore_index=True)
    combined_dt.to_csv(output_name, index=False)

In [193]:
save_prediction(prediction, "prediction_test/hourlyActivity_summary.csv", used_idds, value_scaler, csv_files['hourlyActivity'])

In [203]:
def backtesting(model, series: List[TimeSeries], covar: List[TimeSeries]):
    output = []
    for s, c in zip(series, covar):
        backtest_cov = model.historical_forecasts(s, past_covariates=c, start=0.5, forecast_horizon=1, stride=1, retrain=False, verbose=False)
        output.append(smape(s, backtest_cov))
    return np.mean(output)

In [205]:
mean_smape = backtesting(model, trains, cov_trains)
print('SMAPE (using covariates) = {:.2f}%'.format(average_smape))

SMAPE (using covariates) = 70.95%


# Daily Records

In contrast daily records are rather diverse and cover a variety of different behaviors. This includes sleep, weight as well as activities.
Unfortunately, the amount of subjects who have data across all these areas are very limited. Preventing a complete modeling approach across all variables and subjects.
Hence, I have opted to model each area separately.

## Daily Activity

In [206]:
value_scalar = Scaler()
covar_scalar = Scaler()
ddf_name = 'dailyActivity'
dddf = csv_files[ddf_name]

variables = [k for k in dddf.columns if not any(u in k for u in ("Id", "datetime"))]
ts_series, idds_selected = preprocessing_hourly(dddf, value_scalar, variables, min_req_length=7, fill_missing=True)
ts_series = [fill_missing_values(k) for k in ts_series]  #fill missing values here, while this is not ideal for a production model for this POC it is sufficient
ts_covar = generate_covariates(ts_series, ['weekday', 'month'], covar_scalar)

Using len(idds_timeseries)=29


In [207]:
daily_model_activity = TCNModel(
    input_chunk_length=7,
    output_chunk_length=2,
    n_epochs=20,
    random_state=0,
    torch_device_str="cuda:0",
    # log_tensorboard=True,
    likelihood=GaussianLikelihood(),
    model_name=f"daily_model_{ddf_name}",
    batch_size=2,
    force_reset=True
)

In [208]:
# save the last 7h of each series as test data
train_val = [(k[:-7], k[-7:]) for k in ts_series]
cov_train_val = [(k[:-7], k[-7:]) for k in ts_covar]

In [209]:
trains, vals = zip(*train_val)
cov_trains, cov_vals = zip(*cov_train_val)

In [210]:
daily_model_activity.fit(trains, past_covariates=cov_trains, verbose=True)

  0%|          | 0/20 [00:00<?, ?it/s]

In [211]:
all_covariates =  generate_future_covarites(dddf, "D", len(trains), ['month', 'weekday'], covar_scaler)
prediction = daily_model_activity.predict(2, vals, past_covariates=all_covariates, num_samples=100)

In [212]:
save_prediction(prediction, "prediction_test/dailyActivity_summary.csv", idds_selected, value_scalar, dddf)

In [214]:
mean_smape = backtesting(daily_model_activity, ts_series, ts_covar)
print('SMAPE (using covariates) = {:.2f}%'.format(average_smape))

SMAPE (using covariates) = 70.95%


## Daily Sleep

In [215]:
value_scalar = Scaler()
covar_scalar = Scaler()
ddf_name = 'dailySleep'
dddf = csv_files[ddf_name]

variables = [k for k in dddf.columns if not any(u in k for u in ("Id", "datetime"))]
ts_series, idds_selected = preprocessing_hourly(dddf, value_scalar, variables, min_req_length=7, fill_missing=True)
ts_series = [fill_missing_values(k) for k in ts_series]  #fill missing values here, while this is not ideal for a production model for this POC it is sufficient
ts_covar = generate_covariates(ts_series, ['weekday', 'month'], covar_scalar)

Using len(idds_timeseries)=13


In [217]:
daily_model_sleep = TCNModel(
    input_chunk_length=7,
    output_chunk_length=2,
    n_epochs=20,
    random_state=0,
    torch_device_str="cuda:0",
    # log_tensorboard=True,
    likelihood=GaussianLikelihood(),
    model_name=f"daily_model_{ddf_name}",
    batch_size=2,
    force_reset=True
)

In [218]:
# save the last 7h of each series as test data
train_val = [(k[:-7], k[-7:]) for k in ts_series]
cov_train_val = [(k[:-7], k[-7:]) for k in ts_covar]

In [219]:
trains, vals = zip(*train_val)
cov_trains, cov_vals = zip(*cov_train_val)

In [220]:
daily_model_sleep.fit(trains, past_covariates=cov_trains, verbose=True)

  0%|          | 0/20 [00:00<?, ?it/s]

In [141]:
all_covariates =  generate_future_covarites(dddf, "D", len(trains), ['month', 'weekday'], covar_scaler)
prediction = daily_model.predict(2, vals, past_covariates=all_covariates, num_samples=100)

In [142]:
save_prediction(prediction, "prediction_test/dailySleep_summary.csv", idds_selected, value_scalar, dddf)

In [222]:
mean_smape = backtesting(daily_model_sleep, ts_series, ts_covar)
print('SMAPE (using covariates) = {:.2f}%'.format(average_smape))

SMAPE (using covariates) = 70.95%


## Daily Weight

In [223]:
value_scalar = Scaler()
covar_scalar = Scaler()
ddf_name = 'dailyWeightLog'
dddf = csv_files[ddf_name]

variables = [k for k in dddf.columns if not any(u in k for u in ("Id", "datetime"))]
ts_series, idds_selected = preprocessing_hourly(dddf, value_scalar, variables, min_req_length=7, fill_missing=True)
ts_series = [fill_missing_values(k) for k in ts_series]  #fill missing values here, while this is not ideal for a production model for this POC it is sufficient
ts_covar = generate_covariates(ts_series, ['weekday', 'month'], covar_scalar)

Using len(idds_timeseries)=2


In [228]:
daily_model_sleep = TCNModel(
    input_chunk_length=7,
    output_chunk_length=2,
    n_epochs=20,
    random_state=0,
    torch_device_str="cuda:0",
    # log_tensorboard=True,
    likelihood=GaussianLikelihood(),
    model_name=f"daily_model_{ddf_name}",
    batch_size=2,
    force_reset=True
)

In [229]:
# save the last 7d of each series as test data
train_val = [(k[:-7], k[-7:]) for k in ts_series]
cov_train_val = [(k[:-7], k[-7:]) for k in ts_covar]

In [230]:
trains, vals = zip(*train_val)
cov_trains, cov_vals = zip(*cov_train_val)

In [231]:
daily_model_sleep.fit(trains, past_covariates=cov_trains, verbose=True)

  0%|          | 0/20 [00:00<?, ?it/s]

In [233]:
all_covariates =  generate_future_covarites(dddf, "D", len(trains), ['month', 'weekday'], covar_scaler)
prediction = daily_model_sleep.predict(2, vals, past_covariates=all_covariates, num_samples=100)

In [234]:
save_prediction(prediction, "prediction_test/dailyWeightLog_summary.csv", idds_selected, value_scalar, dddf)

In [235]:
average_smape = backtesting(daily_model_sleep, ts_series, ts_covar)
print('SMAPE (using covariates) = {:.2f}%'.format(average_smape))

SMAPE (using covariates) = 94.92%


# Conclusion

Due to the limited training data a comprehensive model performance analysis is rather difficult.
As indicated by the sMAPE score, current performance is not desirable. However, given the limited sample size as well as the large data requirement of deep learning model these results are not very surprising.